<h1> Black-Litterman Optimization

Black-Litterman Model

$$
\max_{w} \, \mu_{BL}^T w - \frac{\lambda}{2} w^T \Sigma_{BL} w
$$

- λ: Risk aversion coefficient  
- w: Asset allocation weights, \( w^T 1 = 1 \)  
- μ_BL: Expected returns of assets reflecting investor views  
- Σ_BL: Covariance matrix of asset returns reflecting investor views

In [1]:
!pip install PyPortfolioOpt

# coding: utf-8
from typing import Dict, Optional

import numpy as np
import pandas as pd
import plotly.express as px
from numpy.linalg import inv, pinv
from pypfopt import EfficientFrontier

# provided as *.py in directory
from data_loader import PykrxDataLoader
from account import Account
from broker import Broker
from utility import get_lookback_fromdate
from utility import rebalance
from metric import cagr, mdd, sharpe_ratio, sortino_ratio
from visualize import plot_cumulative_return

<h1> Calculate return function

In [11]:
# This calculates the percentage returns based on the closing prices in the OHLCV (Open-High-Low-Close-Volume) data.
def calculate_return(ohlcv_data: pd.DataFrame):
    
    close_data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)
    close_data = close_data['close']

    # Calculate
    return_data = close_data.pct_change(1) * 100

    return return_data


<h1> B-L portfolio weights function

In [12]:

# This applies the Black-Litterman model to calculate portfolio weights, given market data and investor views

def get_black_litterman_weights(return_data: pd.DataFrame,
                                ohlcv_data: pd.DataFrame,
                                views: np.array,               # An array containing investor views on returns
                                relation_matrix: np.array,     # A matrix relating investor views to assets
                                risk_aversion: Optional[float] = None,
                                risk_free_rate: Optional[float] = 0.00) -> Optional[Dict]:
    # 1. Check missing data
    if return_data.isnull().values.any():
        return None

    # 2. Calculates the covariance matrix of the returns data
    covariance = return_data.cov().values
    if np.isnan(covariance).any():
        return None

    # 3. Uses market capitalization to compute the proportionate weights of each asset
    market_weight = ohlcv_data['market_cap'] / sum(ohlcv_data['market_cap'])
    market_weight.index = ohlcv_data['ticker'].to_list()
    ticker_order = return_data.columns
    market_weight = market_weight.reindex(ticker_order)

    # 4. Risk Aversion Coefficient Calculation
    # -- 1) 기대수익률 및 분산 계산
    expected_return = return_data.mean().multiply(market_weight).sum()
    variance = market_weight.T.values @ covariance @ market_weight.values
    # -- 2) 초과수익률 및 위험회피계수 계산
    excess_return = expected_return - risk_free_rate
    risk_aversion_coefficient = (
            excess_return / variance) if risk_aversion is None else risk_aversion
    # -- 3) 위험회피계수가 음수인 경우 핸들링
    if risk_aversion_coefficient < 0:
        print(
            f'risk_aversion_coefficient is negative: {risk_aversion_coefficient}.'
            f'\nTherefore, it is replaced with the value of 2.3.')
        risk_aversion_coefficient = 0.025

    # 5. Calculates equilibrium returns, which represent the "implied" returns from the market.
    equilibrium_returns = risk_aversion_coefficient * covariance.dot(market_weight)

    # 6. Defines an uncertainty matrix (omega) based on investor views, using tau (a scaling parameter) 
    #    and the relation_matrix (P), which reflects how views relate to assets.
    tau = 0.025
    K = len(views)
    P = relation_matrix
    omega = tau * P.dot(covariance).dot(P.T) * np.eye(N=K)

    # 7. Black-Litterman Expected Returns Calculation
    # E(R)=[(τΣ)^(-1)+P^T ΩP]^(-1) [(τΣ)^(-1) Π+P^T ΩQ]
    BL_expected_return = equilibrium_returns + tau * covariance.dot(P.T).dot(
        inv(P.dot(tau * covariance).dot(P.T) + omega).dot(Q - P.dot(equilibrium_returns)))

    # 8. Black-Litterman Covariance Matrix Calculation
    # ΣBL= Σ + [(τΣ)−1+PTΩ−1P]−1
    # ΣBL= Σ + Alpha
    # When using inv() instead of pinv(), Alpha becomes asymmetric.
    Alpha = pinv(pinv(tau * covariance) + P.T @ inv(omega) @ P).round(9)
    BL_covariance = covariance + Alpha
    BL_covariance = pd.DataFrame(data=BL_covariance, index=ticker_order, columns=ticker_order)

    # optimization
    # Initializes an instance of EfficientFrontier with the calculated Black-Litterman expected returns and covariance matrix, 
    # using OSQP as the solver.


    ef = EfficientFrontier(
        expected_returns=BL_expected_return,
        cov_matrix=BL_covariance,
        solver='OSQP'
    )

    ef.max_quadratic_utility(risk_aversion=risk_aversion_coefficient)
    weights = dict(ef.clean_weights(rounding=None))

    return weights


<h1> B-L simulation function

In [86]:
# simulate_black_litterman iterates through each date to rebalance the portfolio based on updated weights 
# calculated using the Black-Litterman model.


def simulate_black_litterman(ohlcv_data: pd.DataFrame,
                             window_length: int,
                             views: np.array,
                             relation_matrix: np.array,
                             risk_aversion: Optional[float] = None,
                             risk_free: Optional[float] = 0.00) -> Account:
    # 1. 계좌 및 브로커 선언
    account = Account(initial_cash=100000000)
    broker = Broker()

    # 2. 수익률 계산
    return_data = calculate_return(ohlcv_data=ohlcv_data)

    for date, ohlcv in ohlcv_data.groupby(['date']):
        print(date)

        # 3. 주문 집행 및 계좌 갱신
        transactions = broker.process_order(dt=date, data=ohlcv,
                                            orders=account.orders)
        account.update_position(transactions=transactions)
        account.update_portfolio(dt=date, data=ohlcv)
        account.update_order()

        # 4. 블랙-리터만 전략을 이용하여 포트폴리오 구성
        
        if isinstance(date, tuple):
           date = date[0]  # Extract the first element if it's a tuple

        if not isinstance(date, pd.Timestamp):
           date = pd.Timestamp(date)  # Ensure it's a Timestamp


        return_data_slice = return_data.loc[:date].iloc[-window_length:, :]  # loc selects by label, iloc selects by position(row numbers)
        weights = get_black_litterman_weights(return_data=return_data_slice,
                                              ohlcv_data=ohlcv,
                                              risk_aversion=risk_aversion,
                                              risk_free_rate=risk_free,
                                              views=views,
                                              relation_matrix=relation_matrix)

        print(f'Portfolio: {weights}')
        if weights is None:
            continue

        # 6. 주문
        rebalance(dt=date, data=ohlcv, account=account, weights=weights)

    return account


<h1> Simulation Example</h1>

In [87]:
# Date
fromdate = '2020-07-10'
todate = '2023-09-27'

# Ticker
ticker_list = ['005930', '000660', '207940', 
               '051910', '006400', '005380', 
               '000270', '005490', '035420']

# Window
window = 10

adj_fromdate = get_lookback_fromdate(fromdate=fromdate, lookback=window, freq='m')

# Load data
data_loader = PykrxDataLoader(fromdate=adj_fromdate, todate=todate, market="KOSPI")
ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list, freq='m', delay=1)

# Market capitalization
market_cap_data = data_loader.load_market_cap_data(
    ticker_list=ticker_list, freq='m', delay=1)


In [88]:
# Merge dataframe
data = pd.merge(ohlcv_data.reset_index(),
                market_cap_data[['market_cap', 'ticker']].reset_index(),
                on=['date', 'ticker']).set_index('date')

data.head()

,ticker,open,high,low,close,volume,market_cap
date,,,,,,,
2019-10-31,000270,41950,43100,41950,42550,1226642,17248210414850
2019-11-30,000270,42150,44800,41800,43250,18030352,17531964757750
2019-12-31,000270,43300,45650,42150,44300,16584597,17957596272100
2020-01-31,000270,44100,44300,39950,40900,24817954,16579360892300
2020-02-29,000270,40200,42300,36000,36250,22325910,14694421328750


In [89]:
return_data = calculate_return(ohlcv_data=ohlcv_data)
print(return_data.index)
print(return_data.index.names)
if isinstance(return_data.index, pd.DatetimeIndex):
    print("The index is a DatetimeIndex.")
else:
    print("The index is NOT a DatetimeIndex.")


DatetimeIndex(['2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31',
               '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31',
               '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30',
               '2020-10-31', '2020-11-30', '2020-12-31', '2021-01-31',
               '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31',
               '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30',
               '2021-10-31', '2021-11-30', '2021-12-31', '2022-01-31',
               '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31',
               '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30',
               '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-31',
               '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31',
               '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30'],
              dtype='datetime64[ns]', name='date', freq=None)
['date']
The index is a DatetimeIndex.


In [90]:
# Risk free rate
risk_free = 0.00
risk_aversion = None

# Investor view matrix
Q = np.array([0.02, 0.03])
P = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, -1, 1]])

# B-L

account = simulate_black_litterman(ohlcv_data=data,
                                   window_length=window,
                                   views=Q,
                                   relation_matrix=P)

(Timestamp('2019-10-31 00:00:00'),)
Portfolio: None
(Timestamp('2019-11-30 00:00:00'),)
Portfolio: None
(Timestamp('2019-12-31 00:00:00'),)
Portfolio: None
(Timestamp('2020-01-31 00:00:00'),)
Portfolio: None
(Timestamp('2020-02-29 00:00:00'),)
Portfolio: None
(Timestamp('2020-03-31 00:00:00'),)
Portfolio: None
(Timestamp('2020-04-30 00:00:00'),)
Portfolio: None
(Timestamp('2020-05-31 00:00:00'),)
Portfolio: None
(Timestamp('2020-06-30 00:00:00'),)
Portfolio: None
(Timestamp('2020-07-31 00:00:00'),)
Portfolio: None
(Timestamp('2020-08-31 00:00:00'),)
Portfolio: {'000270': 0.0, '000660': 0.2612141823172792, '005380': 0.0646417604840226, '005490': 0.040829012500081, '005930': 0.4078422493810178, '006400': 0.0931812774480518, '035420': 0.1322915178695474, '051910': 0.0, '207940': 0.0}
(Timestamp('2020-09-30 00:00:00'),)
Portfolio: {'000270': 0.0, '000660': 0.2141460068170406, '005380': 0.0595347678306836, '005490': 0.0320605585374704, '005930': 0.466587237910671, '006400': 0.08067345250073

<h1> Simulation analysis</h1>

In [ ]:
df_account = pd.DataFrame(account.account_history).set_index('date')
df_portfolio = pd.DataFrame(account.portfolio_history).set_index('date')
analysis_fromdate = df_account.index[window]

In [ ]:
returns = df_account['total_asset'].pct_change().loc[analysis_fromdate:]
returns.name = 'return'
returns.head()

In [ ]:
kospi = data_loader.load_index_data(ticker_list=['1001'], freq='m', delay=1)
kospi_returns = kospi['close'].pct_change().loc[analysis_fromdate:]
kospi_returns.iloc[0] = 0.0
kospi_returns.name = 'kospi_return'
kospi_returns.index.name = 'date'
kospi_returns.head()

In [ ]:
cagr(returns=kospi_returns, freq='m')
mdd(returns=kospi_returns)
sharpe_ratio(returns=returns, freq='m')
sortino_ratio(returns=returns, freq='m')

In [ ]:
plot_cumulative_return(returns=returns, benchmark_returns=kospi_returns,
                       strategy_name='블랙-리터만',
                       benchmark_name='코스피 지수')